In [59]:
import pandas as pd
import json

In [60]:
results = {}
project = "Plants"
methods = ["Mo2oM_Structural", "Mo2oM_UniXcoder", "Mo2oM_Combined"]
t_range = {"Mo2oM_Structural": [0.15, 0.20], "Mo2oM_UniXcoder": [0.10, 0.25], "Mo2oM_Combined": [0.05, 0.20]}
metrics = ["SM", "ICP", "IFN", "NED"]
metrics_sign = {"SM": 1, "ICP": -1, "IFN": -1, "NED": -1}

for method in methods:
	results[method] = {}
	with open(f"../results/{method}/{method}_{project}.json", "r") as f:
		data = json.load(f)
		df = pd.DataFrame(data).drop(["microservices", "n_clusters"], axis=1)
		df = df[df["threshold"].between(t_range[method][0], t_range[method][1])]
		for metric in metrics:
				# if metrics_sign[metric] == 1:
				# 	results[project][method][metric] = df[metric].max()
				# else:
				# 	results[project][method][metric] = df[metric].min()
				results[method][metric] = df[metric].median()

In [61]:
bests = {}
for metric in metrics:
	best_method = None
	best_metric = float('-inf') * metrics_sign[metric]
	for method in methods:
		if metrics_sign[metric] == 1:
			if results[method][metric] > best_metric:
				best_metric = results[method][metric]
				best_method = method
		else:
			if results[method][metric] < best_metric:
				best_metric = results[method][metric]
				best_method = method
	bests[metric] = best_method

In [62]:
bests

{'SM': 'Mo2oM_Combined',
 'ICP': 'Mo2oM_Combined',
 'IFN': 'Mo2oM_Combined',
 'NED': 'Mo2oM_UniXcoder'}

In [63]:
table_head = r'''	\begin{table}[ht]
		\centering
		\begin{tabular}{|l|c|c|c|c|}
			\hline
			\textbf{Features} & \textbf{SM $\uparrow$} & \textbf{ICP $\downarrow$} & \textbf{IFN $\downarrow$} & \textbf{NED $\downarrow$} \\
			\hline'''
table_tail = r'''			\hline
		\end{tabular}
		\caption{ project }
	\end{table}'''

In [64]:
print(table_head)
for method in methods:
	line = "	"*3 + method.replace("Mo2oM_", "")
	for metric in metrics:
		if bests[metric] == method:
			line += f" & \\textbf{{{results[method][metric]:.3f}}}"
		else:
			line += f" & {results[method][metric]:.3f}"
	line += r"\\"
	print(line)
print(table_tail.replace("project", project))

	\begin{table}[ht]
		\centering
		\begin{tabular}{|l|c|c|c|c|}
			\hline
			\textbf{Features} & \textbf{SM $\uparrow$} & \textbf{ICP $\downarrow$} & \textbf{IFN $\downarrow$} & \textbf{NED $\downarrow$} \\
			\hline
			Structural & 0.371 & 0.098 & 0.938 & 0.427\\
			UniXcoder & 0.449 & 0.101 & 1.275 & \textbf{0.286}\\
			Combined & \textbf{0.532} & \textbf{0.056} & \textbf{0.417} & 0.509\\
			\hline
		\end{tabular}
		\caption{ Plants }
	\end{table}
